# Multiple-objective Knapsack Problem with amplpy
[![bpp.ipynb](https://img.shields.io/badge/github-%23121011.svg?logo=github)](https://github.com/ampl/colab.ampl.com/blob/master/authors/lentz/multiobj/knapsack.ipynb) [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ampl/colab.ampl.com/blob/master/authors/lentz/multiobj/knapsack.ipynb) [![Open In Deepnote](https://deepnote.com/buttons/launch-in-deepnote-small.svg)](https://deepnote.com/launch?url=https://github.com/ampl/colab.ampl.com/blob/master/authors/lentz/multiobj/knapsack.ipynb) [![Open In Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/ampl/colab.ampl.com/blob/master/authors/lentz/multiobj/knapsack.ipynb) [![Open In Gradient](https://assets.paperspace.io/img/gradient-badge.svg)](https://console.paperspace.com/github/ampl/colab.ampl.com/blob/master/authors/lentz/multiobj/knapsack.ipynb) [![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/ampl/colab.ampl.com/blob/master/authors/lentz/multiobj/knapsack.ipynb) [![Powered by AMPL](https://h.ampl.com/https://github.com/ampl/colab.ampl.com/blob/master/authors/lentz/multiobj/knapsack.ipynb)](https://ampl.com)

Description: Knapsack Problem using Multiple Objectives

Tags: multiple-objectives, knapsack, amplpy, highlights

Notebook author: Jurgen Lentz <<jurgenlentz26@gmail.com>>

This notebook demonstrates how to solve a **multiple-objective knapsack problem** via `amplpy`.
We will:
- Set up a knapsack model with two objectives (e.g., maximize profit & minimize weight).
- Compute the **Pareto frontier**.
- Visualize trade-offs between objectives.

By the end, you’ll understand how to use AMPL for multi-objective optimization and interpret Pareto‐optimal solutions.

In [17]:
# Install dependencies
%pip install -q amplpy

Note: you may need to restart the kernel to use updated packages.


In [18]:
# Google Colab & Kaggle integration
from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["gurobi", "highs", "cbc"],  # modules to install
    license_uuid="default",  # license to use
)  # instantiate AMPL object and register magics

AMPL Version 20250515 (Linux-6.8.0-1027-azure, 64-bit)
Demo license with maintenance expiring 20270131.
Using license file "/home/lentz/.local/lib/python3.10/site-packages/ampl_module_base/bin/ampl.lic".



## Instantiate the AMPL environment

This cell initializes an AMPL session in amplpy, sets the solver to Gurobi but you can choose another solver (e.g., HiGHS, CBC), and activates the MP multi-objective emulation (you can set obj:multi to 1 if you want to use multi-objective functionalities of the solver).

In [19]:
from amplpy import AMPL

# Initialize AMPL
ampl = AMPL()

ampl.setOption("solver", "gurobi")  # or 'highs' or 'cbc'
ampl.setOption("gurobi_options", "obj:multi=2")  # enable lexicographic MO support


We create the binary knapsack model but use two different objective functions, `TotalValue` to maximize the total profits of the items packed in knapsack and `NumItems` to minimize the number of items packed in the knapsack. The suffix `objpriority` specifies the lexicographical order of the objective functions.  

In [20]:
%%ampl_eval
suffix objpriority;
suffix option_timelimit;
suffix option_mipgap;

set ITEMS;
param weight{ITEMS};
param value{ITEMS};
param capacity;

var x{ITEMS} binary;

# Objective 1: maximize total value
maximize TotalValue:
    sum {i in ITEMS} value[i] * x[i]
    suffix objpriority 2;  # highest priority

# Objective 2: minimize number of items
minimize NumItems:
    sum {i in ITEMS} x[i]
    suffix objpriority 1;

subject to CapacityConstraint:
    sum {i in ITEMS} weight[i] * x[i] <= capacity;

We provide data for the set `ITEMS` and assign corresponding values to the parameters `weight`, `value`, and `capacity`.

In [ ]:
items = ["item1", "item2", "item3", "item4", "item5"]
weights = [2, 3, 4, 5, 9]
values = [3, 4, 8, 8, 10]
capacity = 10

ampl.set["ITEMS"] = items
ampl.param["weight"] = dict(zip(items, weights))
ampl.param["value"] = dict(zip(items, values))
ampl.param["capacity"] = capacity

You can specify options for each objective by creating a suffix in AMPL with the name starting with `option_` followed by the option name as obtained by the solver. Here, we set the `timelimit` and `mipgap` of the objective `TotalValue`.

In [ ]:
ampl.eval(
    """
    let TotalValue.option_timelimit := 60;
    let TotalValue.option_mipgap := 0.01;
    """
)

In [23]:
# Solve
ampl.solve()

Gurobi 12.0.2:   obj:multi = 2
Gurobi 12.0.2: optimal solution; objective 16
Individual objective values:
	_sobj[1] = 16
	_sobj[2] = 2
0 simplex iterations
Objective = TotalValue


In [24]:
# Retrieve and display results
tv = ampl.getObjective("TotalValue").value()
ni = sum(int(ampl.getVariable("x")[i].value()) for i in items)
sel = [i for i in items if int(ampl.getVariable("x")[i].value()) == 1]

print(f"🏅 TotalValue = {tv}")
print(f"🧩 NumItems = {ni}")
print("✅ Selected items:", sel)

🏅 TotalValue = 16.0
🧩 NumItems = 2
✅ Selected items: ['item3', 'item4']
